т.к. маркеры погоды не пересекаются, то имеет смысл обучить отдельный алгоритм для этих маркеров, а для остальных использовать другие алгоритмы

библиотеки

In [29]:
import numpy as np
import pandas as pd
import os
from natsort import natsorted, ns

import keras
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

import cv2
import imutils
from tqdm import tqdm

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder

загрузка и обработка данных

In [3]:
def PathDetermination(user):
    if user == 'litvin_home':
        core_dir = 'C:\\Kaggle\\Understanding the Amazon from Space\\'
    
    if user == 'litvin_office':
        core_dir = 'D:\\Kaggle\\Understanding the Amazon from Space\\'
        train_dir = core_dir + 'train-jpg\\'
        test_dir = core_dir + 'test-jpg\\'
        add_test_dir = core_dir + 'test-jpg-additional\\'
    
    if user == 'savina':
        core_dir = 'Z:\\Kaggle Amazon Rainforest\\'
        train_dir = ''
        test_dir = 'C:\\Users\\horch\\Desktop\\Local_data\\test-jpg\\'
        add_test_dir = 'C:\\Users\\horch\\Desktop\\Local_data\\test-jpg-additional\\'
    
    return core_dir, train_dir, test_dir, add_test_dir

In [12]:
core_dir, train_dir, test_dir, add_test_dir = PathDetermination('litvin_office')
#core_dir, test_dir, add_test_dir = path_determination('savina')

test_data_names = natsorted(os.listdir(test_dir), key=lambda y: y.lower())
add_test_data_names = os.listdir(add_test_dir)


cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

weather_cathegories = ['clear', 'cloudy', 'haze', 'partly_cloudy']

train_data = pd.read_csv(core_dir + 'train_v2.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


функции

In [46]:
def BinimialPrediction(X, treshold = 0.5):
    result = np.zeros(X.shape)
    
    if type(treshold) == int or type(treshold) == float:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                if X[i, j] >= treshold:
                    result[i, j] = 1
    elif type(treshold) == list:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                if X[i, j] >= treshold[j]:
                    result[i, j] = 1
    else:
        print('treshold type must be int, float or list')
    
    return(result)

def FBettaScore(x_true, x_predicted, betta = 2):
    if len(x_true) == len(x_predicted):
        tp = 0
        fp = 0
        fn = 0

        for i in range(len(x_predicted)):
            if x_true[i] == 1 and x_predicted[i] == 1:
                tp += 1
            
            if x_true[i] == 0 and x_predicted[i] == 1:
                fp += 1
            
            if x_true[i] == 1 and x_predicted[i] == 0:
                fn += 1
        
        if tp == 0 or (tp + fp) == 0 or (tp + fn) == 0:
            return(0)
        else:
            precision = tp/(tp + fp)
            recall = tp/(tp + fn)
            
            return((1 + betta**2)*precision*recall/(betta**2*precision + recall))
    else:
        print('FBettaScore error! len(x_true) != len(x_predicted)')

def AvgFBettaScore(x_true, x_predicted, betta = 2, treshold = 0.5):
    result = 0
    n = x_true.shape[0]
    
    x_predicted = BinimialPrediction(x_predicted, treshold)
    
    for i in range(n):
        result += FBettaScore(x_true[i, :], x_predicted[i, :], betta)
    
    return(result/n)

вычисления

In [9]:
def DataPreperation(data_type, img_size = (32, 32), rotation = [0], 
                    test_img_dir = [test_dir, add_test_dir], shuffle = False):
    if data_type == 'train':
        X = []
        Y = []
        
        for img_name in tqdm(train_data.image_name.values):
    
            img = cv2.imread(train_dir + img_name + '.jpg')
            img_resized = cv2.resize(img, img_size)
            
            img_tags = train_data[train_data['image_name'] == img_name]['tags'].values[0].split(' ')
            y = np.zeros(len(cathegories))
            
            for i in range(len(y)):
                if cathegories[i] in img_tags:
                    y[i] = 1
            
            for angle in rotation:
                img = imutils.rotate(img_resized, angle)
                X.append(img)
                Y.append(y) 
        
        X = np.array(X, np.float16) / 255.
        Y = np.array(Y)
        
        if shuffle:
            ind = np.random.choice(np.arange(X.shape[0]), X.shape[0], replace= False)
            X = X[ind]
            Y = Y[ind]
        
        return X, Y
    
    if data_type == 'test':
        X_test = []
        X_test_names = []

        for img_dir in test_img_dir:
            img_dir_names = os.listdir(img_dir)
            
            for img_name in tqdm(img_dir_names):
                if img_name.endswith("jpg"):
                    img = cv2.imread(img_dir + img_name)
                    img = cv2.resize(img, img_size)
                    
                    X_test.append(img)
                    X_test_names.append(img_name)
    
        X_test = np.array(X_test, np.float16) / 255.
        
        return X_test, X_test_names

In [10]:
X, Y = DataPreperation('train', rotation = [0], shuffle= True)

100%|████████████████████████████████████| 40479/40479 [12:23<00:00, 54.41it/s]


In [18]:
Y_weather = Y[:, [cathegories.index(x) for x in weather_cathegories]]

In [37]:
sss = StratifiedShuffleSplit(n_splits= 1, test_size= 0.3, random_state= 42)

for train_index, test_index in sss.split(X, Y_weather):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y_weather[train_index], Y_weather[test_index]

Настройка и  обучение сети для погоды

In [23]:
batch_size = 128
num_epochs = 50
kernel_size = 3
pool_size = 2
conv_depth_1 = 32
conv_depth_2 = 64
drop_prob_1 = 0.25
drop_prob_2 = 0.5
hidden_size = 512

In [25]:
num_train = X.shape[0]
depth = 32 
height = 32
width = 3 

num_classes = len(weather_cathegories)

In [52]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)

conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)

conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax

flat = Flatten()(drop_2)

hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)

out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(input=inp, output=out)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [53]:
#val_acc: 0.9401

model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=num_epochs, verbose=1, validation_data = (x_test, y_test), 
          class_weight= [1, 10, 5, 2])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 28335 samples, validate on 12144 samples
Epoch 1/50
28335/28335 [==============================] - 162s - loss: 0.6871 - acc: 0.7319 - val_loss: 0.5886 - val_acc: 0.7918
Epoch 2/50
28335/28335 [==============================] - 161s - loss: 0.4962 - acc: 0.8156 - val_loss: 0.4926 - val_acc: 0.8145
Epoch 3/50
28335/28335 [==============================] - 163s - loss: 0.3980 - acc: 0.8564 - val_loss: 0.3817 - val_acc: 0.8593
Epoch 4/50
28335/28335 [==============================] - 162s - loss: 0.3479 - acc: 0.8763 - val_loss: 0.3398 - val_acc: 0.8745
Epoch 5/50
28335/28335 [==============================] - 161s - loss: 0.3087 - acc: 0.8884 - val_loss: 0.5131 - val_acc: 0.8152
Epoch 6/50
28335/28335 [==============================] - 160s - loss: 0.2963 - acc: 0.8925 - val_loss: 0.2804 - val_acc: 0.8954
Epoch 7/50
28335/28335 [==============================] - 161s - loss: 0.2717 - acc: 0.8999 - val_loss: 0.2818 - val_acc: 0.8953
Epoch 8/50
28335/28335 [=======================

In [54]:
weather_nn_model_save_dir = 'weather nn//'
model_name = '50e + weights 1-10-5-2'

model_json = model.to_json()

json_file = open(weather_nn_model_save_dir + model_name + ".json", "w")
json_file.write(model_json)
json_file.close()

model.save_weights(weather_nn_model_save_dir + model_name + ".h5")

анализ результата

In [55]:
x_test_predicted = model.predict(x_test)
x_test_real = y_test

In [69]:
def PredictMax(x):
    x_predicted = np.zeros(x.shape)
    
    for i in range(x.shape[0]):
        ind = np.argmax(x[i, :])
        x_predicted[i, ind] = 1
    
    return x_predicted

In [63]:
# 0.09 0.9545518264947699
# 0.1 0.9575716403162146 with weights
'''
fscore = 0

for tres in np.arange(0, 1, 0.01):
    cscore = AvgFBettaScore(x_test_real, x_test_predicted, treshold= float(tres))
    
    if cscore > fscore:
        fscore = cscore
        print(tres, fscore)

'''

'\nfscore = 0\n\nfor tres in np.arange(0, 1, 0.01):\n    cscore = AvgFBettaScore(x_test_real, x_test_predicted, treshold= float(tres))\n    \n    if cscore > fscore:\n        fscore = cscore\n        print(tres, fscore)\n\n'

In [71]:
x_test_binomial = PredictMax(x_test_predicted)

errors = np.zeros((len(weather_cathegories), 4))

for i in range(x_test_binomial.shape[0]):
    for j in range(x_test_binomial.shape[1]):
        predicted = x_test_binomial[i, j]
        real = x_test_real[i, j]
        
        
        if predicted == 1 and real == 1:
            errors[j, 0] += 1
        
        if predicted == 1 and real == 0:
            errors[j, 1] += 1
        
        if predicted == 0 and real == 1:
            errors[j, 2] += 1
        
        if real == 1:
            errors[j, 3] += 1

errors = errors.astype(int)
errors_df = pd.DataFrame(errors, index= weather_cathegories, columns= ['tp', 'fp', 'fn', 'cnt'])
errors_df

,tp,fp,fn,cnt
clear,8267,320,182,8449
cloudy,521,70,140,661
haze,607,174,236,843
partly_cloudy,2044,141,146,2190


In [72]:
x_test_binomial = BinimialPrediction(x_test_predicted, treshold= 0.2)

errors = np.zeros((len(weather_cathegories), 4))

for i in range(x_test_binomial.shape[0]):
    for j in range(x_test_binomial.shape[1]):
        predicted = x_test_binomial[i, j]
        real = x_test_real[i, j]
        
        
        if predicted == 1 and real == 1:
            errors[j, 0] += 1
        
        if predicted == 1 and real == 0:
            errors[j, 1] += 1
        
        if predicted == 0 and real == 1:
            errors[j, 2] += 1
        
        if real == 1:
            errors[j, 3] += 1

errors = errors.astype(int)
errors_df = pd.DataFrame(errors, index= weather_cathegories, columns= ['tp', 'fp', 'fn', 'cnt'])
errors_df

,tp,fp,fn,cnt
clear,8354,460,95,8449
cloudy,607,152,54,661
haze,687,370,156,843
partly_cloudy,2080,189,110,2190
